In [229]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side  
from openpyxl.utils import get_column_letter
from datetime import date, timedelta


Overall Guide to the Code:

1) read in Bicycle Data, 
2) prepare a report of the busiest and slowest days of the year to date
3) prepare a report of the daily detail for current week
4) export all data to date

In [230]:
%%script False

#wb = openpyxl.load_workbook('Seoul Bicycle Data.xlsx')

#sheetnames = list(wb.sheetnames)
#print(sheetnames)
#   ['Seoul bicycle rental dataset']

#ws = wb.active

Couldn't find program: 'False'


In [231]:
daily_data = pd.read_excel('Seoul Bicycle Data.xlsx')
#print(daily_data.head())

In [232]:
#daily_data.info()

daily_data['Date'] = pd.to_datetime(daily_data['Date'])
daily_data['Date'] = daily_data['Date'].dt.date

daily_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      8760 non-null   object 
 1   Rented Bike Count         8760 non-null   int64  
 2   Hour                      8760 non-null   int64  
 3   Temperature(蚓)            8760 non-null   float64
 4   Humidity(%)               8760 non-null   int64  
 5   Wind speed (m/s)          8760 non-null   float64
 6   Visibility (10m)          8760 non-null   int64  
 7   Dew point temperature(蚓)  8760 non-null   float64
 8   Solar Radiation (MJ/m2)   8760 non-null   float64
 9   Rainfall(mm)              8760 non-null   float64
 10  Snowfall (cm)             8760 non-null   float64
 11  Seasons                   8760 non-null   object 
 12  Holiday                   8760 non-null   object 
 13  Functioning Day           8760 non-null   object 
dtypes: float

In [233]:
daily_data['Date'].unique()

array([datetime.date(2017, 1, 12), datetime.date(2017, 2, 12),
       datetime.date(2017, 3, 12), datetime.date(2017, 4, 12),
       datetime.date(2017, 5, 12), datetime.date(2017, 6, 12),
       datetime.date(2017, 7, 12), datetime.date(2017, 8, 12),
       datetime.date(2017, 9, 12), datetime.date(2017, 10, 12),
       datetime.date(2017, 11, 12), datetime.date(2017, 12, 12),
       datetime.date(2017, 12, 13), datetime.date(2017, 12, 14),
       datetime.date(2017, 12, 15), datetime.date(2017, 12, 16),
       datetime.date(2017, 12, 17), datetime.date(2017, 12, 18),
       datetime.date(2017, 12, 19), datetime.date(2017, 12, 20),
       datetime.date(2017, 12, 21), datetime.date(2017, 12, 22),
       datetime.date(2017, 12, 23), datetime.date(2017, 12, 24),
       datetime.date(2017, 12, 25), datetime.date(2017, 12, 26),
       datetime.date(2017, 12, 27), datetime.date(2017, 12, 28),
       datetime.date(2017, 12, 29), datetime.date(2017, 12, 30),
       datetime.date(2017, 12, 31)

In [234]:
daily_data['Day'] = pd.to_datetime(daily_data['Date']).dt.day_name()

daily_data['Day'].unique()

array(['Thursday', 'Sunday', 'Wednesday', 'Friday', 'Monday', 'Saturday',
       'Tuesday'], dtype=object)

In [235]:
# get the total rentals by day and get top 10 busiest and slowest days
daily_volume = daily_data.groupby(['Date', 'Day', 'Holiday'])['Rented Bike Count'].sum().sort_values(ascending=False).reset_index()

highest_10 = daily_volume.head(10)
lowest_10 = daily_volume[(daily_volume['Rented Bike Count']) > 0  & (daily_volume['Holiday'] != 'Holiday')].tail(10)

print(lowest_10.info())


<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 343 to 352
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Date               10 non-null     object
 1   Day                10 non-null     object
 2   Holiday            10 non-null     object
 3   Rented Bike Count  10 non-null     int64 
dtypes: int64(1), object(3)
memory usage: 400.0+ bytes
None


In [236]:
# create summary of highest and lowest dates

highest_10_sum = highest_10['Rented Bike Count'].sum()/len(highest_10)
highest_10_avg = (highest_10['Rented Bike Count'].sum()/10)/9

lowest_10_sum = lowest_10['Rented Bike Count'].sum()/len(lowest_10)
lowest_10_avg = (lowest_10['Rented Bike Count'].sum()/10)/9

In [237]:
# create WB and Sheet
wb = openpyxl.Workbook()
ws = wb.active

Make the Busiest and Slowest Day Summary

In [238]:
#format the summary tab

# set the title area
ws.title = 'Summary'

ws['A1'] = 'Seoul Bicycle Rental - Overall Summary Report'
ws['A1'].font = Font(size=16, bold=True)

ws['A2'] = 'Ten 10 Busiest / Slowest Days'
ws['A2'].font = Font(size=12)

data_as_of = '2018-04-14'
ws['A4'] = 'Data as of:'
ws['B4'] = data_as_of


# set the busiest days summary

# set extremes_border
extremes_border = Border(
    top=Side(border_style="thin"),
    bottom=Side(border_style="thin"),
    left=Side(border_style="thin"),
    right=Side(border_style="thin")
)

ws['A7'] = 'Busiest Days To-Date (All Days)'
ws['A7'].font = Font(size=12, bold=True)
ws["A7"].alignment = Alignment(horizontal="center")
ws['A7'].fill = PatternFill(fill_type='solid', start_color='FF70AD47')
ws['A7'].border = extremes_border
ws.merge_cells("A7:E7")


busy_slow_lables = ['Index', 'Date', 'Day of Week', 'Total Rentals', 'Average Hourly Rental']

for col, value in enumerate(busy_slow_lables, start=1):
    
    ws.cell(row=8, column=col, value=value)
    ws.cell(row=8, column=col).fill = PatternFill(fill_type='solid', start_color='FFE2EFDA')
    ws.cell(row=8, column=col).alignment = Alignment(horizontal="center")
    ws.cell(row=8, column=col).border = extremes_border

    
    # the labels for busiest days should never change due to report template; slowest days will be a function of whether the top days is different from standard of 10 days

busiest_total_row = 8 + len(highest_10) + 2

# set totals border
totals_border = Border(
    top=Side(border_style="thin"),
    bottom=Side(border_style="double")
)

ws.cell(row=busiest_total_row, column=3, value='Average Total')
ws.cell(row=busiest_total_row, column=3).alignment = Alignment(horizontal="center")
ws.cell(row=busiest_total_row, column=3).border = totals_border
ws.cell(row=busiest_total_row, column=4, value=highest_10_sum)
ws.cell(row=busiest_total_row, column=4).alignment = Alignment(horizontal='center')
ws.cell(row=busiest_total_row, column=4).number_format = "#,##0"
ws.cell(row=busiest_total_row, column=4).border= totals_border
ws.cell(row=busiest_total_row, column=5, value=highest_10_avg)
ws.cell(row=busiest_total_row, column=5).alignment = Alignment(horizontal='center')
ws.cell(row=busiest_total_row, column=5).number_format = "#,##0"
ws.cell(row=busiest_total_row, column=5).border= totals_border


# set slowest days summary
slowest_row = busiest_total_row + 3

ws['A' + str(slowest_row)] = 'Slowest Days To-Date (non-Holiday)'
ws['A' + str(slowest_row)].font = Font(size=12, bold=True)
ws['A' + str(slowest_row)].alignment = Alignment(horizontal="center")
ws['A' + str(slowest_row)].fill = PatternFill(fill_type='solid', start_color='FFA5A5A5')
ws['A' + str(slowest_row)].border = extremes_border
ws.merge_cells('A' + str(slowest_row) + ':E' + str(slowest_row))

for col, value in enumerate(busy_slow_lables, start=1):
    ws.cell(row=slowest_row+1, column=col, value=value)
    ws.cell(row=slowest_row+1, column=col).fill = PatternFill(fill_type='solid', start_color='FFEDEDED')
    ws.cell(row=slowest_row+1, column=col).alignment = Alignment(horizontal="center")
    ws.cell(row=slowest_row+1, column=col).border = extremes_border

slowest_total_row = slowest_row + len(lowest_10) + 4

ws.cell(row=slowest_total_row, column=3, value='Average Total')
ws.cell(row=slowest_total_row, column=3).alignment = Alignment(horizontal="center")
ws.cell(row=slowest_total_row, column=3).border= totals_border
ws.cell(row=slowest_total_row, column=4, value=lowest_10_sum)
ws.cell(row=slowest_total_row, column=4).alignment = Alignment(horizontal='center')
ws.cell(row=slowest_total_row, column=4).number_format = "#,##0"
ws.cell(row=slowest_total_row, column=4).border= totals_border
ws.cell(row=slowest_total_row, column=5, value=lowest_10_avg)
ws.cell(row=slowest_total_row, column=5).alignment = Alignment(horizontal='center')
ws.cell(row=slowest_total_row, column=5).number_format = "#,##0"
ws.cell(row=slowest_total_row, column=5).border= totals_border



In [239]:
# create the busiest days table

###
busiest_days_df = pd.DataFrame()
### do i need this?

# need to dynamically set the row for the slowest days since we could reset to top and lowest 5 instead of 10
for num in range(0, len(highest_10)):
    ws.cell(row=num + 9, column=1, value=num+1)
    ws.cell(row=num + 9, column=2, value=highest_10.iloc[num]['Date'])
    ws.cell(row=num + 9, column=3, value=highest_10.iloc[num]['Day'])
    ws.cell(row=num + 9, column=4, value=highest_10.iloc[num]['Rented Bike Count'])
    ws.cell(row=num + 9, column=5, value=highest_10.iloc[num]['Rented Bike Count'] / 9)

    # number formatting
    ws.cell(row=num + 9, column=2).number_format = "YYYY-MM-DD"
    ws.cell(row=num + 9, column=4).number_format = "#,##0"
    ws.cell(row=num + 9, column=5).number_format = "#,##0"

    for cell in ws[num + 9]:
        cell.alignment = Alignment(horizontal="center")
   
    # border formatting
    ws.cell(row=num + 9, column=1).border = extremes_border
    ws.cell(row=num + 9, column=2).border = extremes_border
    ws.cell(row=num + 9, column=3).border = extremes_border
    ws.cell(row=num + 9, column=4).border = extremes_border
    ws.cell(row=num + 9, column=5).border = extremes_border


In [240]:
# create the slowest days table

###
slowest_days_df = pd.DataFrame()
###

slowest_row_data = slowest_row + 2

# need to dynamically set the row for the slowest days since we could reset to top and lowest 5 instead of 10
for num in range(0, len(lowest_10)):
    ws.cell(row=slowest_row_data + num, column=1, value=num+1)
    ws.cell(row=slowest_row_data + num, column=2, value=lowest_10.iloc[num]['Date'])
    ws.cell(row=slowest_row_data + num, column=3, value=lowest_10.iloc[num]['Day'])
    ws.cell(row=slowest_row_data + num, column=4, value=lowest_10.iloc[num]['Rented Bike Count'])
    ws.cell(row=slowest_row_data + num, column=5, value=lowest_10.iloc[num]['Rented Bike Count'] / 9)

    # number formatting
    ws.cell(row=slowest_row_data + num, column=2).number_format = "YYYY-MM-DD"
    ws.cell(row=slowest_row_data + num, column=4).number_format = "#,##0"
    ws.cell(row=slowest_row_data + num, column=5).number_format = "#,##0"

    for cell in ws[slowest_row_data + num]:
        cell.alignment = Alignment(horizontal="center")

    # border formatting
    ws.cell(row=slowest_row_data + num, column=1).border = extremes_border
    ws.cell(row=slowest_row_data + num, column=2).border = extremes_border
    ws.cell(row=slowest_row_data + num, column=3).border = extremes_border
    ws.cell(row=slowest_row_data + num, column=4).border = extremes_border
    ws.cell(row=slowest_row_data + num, column=5).border = extremes_border

In [241]:
# set column widths
ws.column_dimensions['A'].width = 10

for col in ws.columns:
    max_length = max(len(str(cell.value)) for cell in col if cell.value)
    col_letter = get_column_letter(col[0].column)
    
    if get_column_letter(col[0].column) != 'A':
        ws.column_dimensions[col_letter].width = max_length + 2


wb.save('Seoul Test.xlsx')

Make Daily Summary for Each Day of the Week

In [242]:
###
### SET THE END OF WEEK DATE ###
##

week_end_date = '2018-04-14'

# Convert to datetime object
week_end_date = datetime.strptime(week_end_date, "%Y-%m-%d").date()


# calculate previous week end date
for i in range(1,9):
    trial_date = (week_end_date) - timedelta(days=i)
    #print(f"The new test date is {trial_date}")
    if trial_date.strftime("%A") == 'Saturday':
        previous_week_end = trial_date

        #print(f"We have found the new date, it is {trial_date}")
        break
    else:
        continue

# calculate upcoming week end date
for i in range(1,9):
    trial_date = week_end_date + timedelta(days=i)
    #print(f"The new test date is {trial_date}")
    if trial_date.strftime("%A") == 'Saturday':
        upcoming_week_end = trial_date
       #print(f"We have found the new date, it is {trial_date}")
        break
    else:
        continue

# validate current week end date      
if week_end_date.strftime("%A") != 'Saturday':
    print(f"The end of week day set is {week_end_date}, which is a {week_end_date.strftime("%A")}. \nPlease correct this date to the actual week ending date you require. \nWhat about {previous_week_end} or {upcoming_week_end}?")
else:
    print(f"The end of week day set is {week_end_date}, which is a {week_end_date.strftime("%A")}. Good to go.")

# set start of week date to begin the data pull for all days in the week
for i in range(1,9):
    trial_date = week_end_date - timedelta(days=i)
    #print(f"The new test date is {trial_date}")
    if trial_date.strftime("%A") == 'Sunday':
        week_start_date = trial_date
        #print(f"We have found the new date, it is {trial_date}")
        break
    else:
        continue

The end of week day set is 2018-04-14, which is a Saturday. Good to go.


In [243]:
# pull the daily data for the week
week_list = []

for i in range((week_end_date - week_start_date).days + 1):
    week_list.append(week_start_date + timedelta(days=i))



week_data = daily_data[daily_data['Date'].isin(week_list)]
week_detail = week_data.groupby(['Date', 'Day', 'Holiday'])['Rented Bike Count'].sum().reset_index()

print(week_detail)

         Date        Day     Holiday  Rented Bike Count
0  2018-04-08     Sunday  No Holiday              18363
1  2018-04-09     Monday  No Holiday              29529
2  2018-04-10    Tuesday  No Holiday                  0
3  2018-04-11  Wednesday  No Holiday              20471
4  2018-04-13     Friday  No Holiday              23240
5  2018-04-14   Saturday  No Holiday               6980


In [244]:
#format the Daily Detail tab

# set the title area
ws = wb.create_sheet(title="Week Detail")

ws['A1'] = 'Seoul Bicycle Rental - Week Detail Report'
ws['A1'].font = Font(size=16, bold=True)


ws['A2'] = 'Daily Detail for Week Ending ' + str(week_end_date)
ws['A2'].font = Font(size=12)


# set the daily detail output

# set extremes_border
# extremes_border = Border(
#     top=Side(border_style="thin"),
#     bottom=Side(border_style="thin"),
#     left=Side(border_style="thin"),
#     right=Side(border_style="thin")
# )

ws['A6'] = 'Daily Detail'
ws['A6'].font = Font(size=12, bold=True)
ws["A6"].alignment = Alignment(horizontal="center")
ws['A6'].fill = PatternFill(fill_type='solid', start_color='FF5B9BD5')
ws['A6'].border = extremes_border
ws.merge_cells("A6:F6")


daily_detail_lables = ['Index', 'Date', 'Day of Week', 'Total Rentals', 'Average Hourly Rental', 'Holiday']

for col, value in enumerate(daily_detail_lables, start=1):
    
    ws.cell(row=7, column=col, value=value)
    ws.cell(row=7, column=col).fill = PatternFill(fill_type='solid', start_color='FFDDEBF7')
    ws.cell(row=7, column=col).alignment = Alignment(horizontal="center")
    ws.cell(row=7, column=col).border = extremes_border


week_detail_start = 8

# populate week detail data
for num in range(0, len(week_detail)):
    ws.cell(row=week_detail_start + num, column=1, value=num+1)
    ws.cell(row=week_detail_start + num, column=2, value=week_detail.iloc[num]['Date'])
    ws.cell(row=week_detail_start + num, column=3, value=week_detail.iloc[num]['Day'])
    ws.cell(row=week_detail_start + num, column=4, value=week_detail.iloc[num]['Rented Bike Count'])
    ws.cell(row=week_detail_start + num, column=5, value=week_detail.iloc[num]['Rented Bike Count'] / 9)
    ws.cell(row=week_detail_start + num, column=6, value=week_detail.iloc[num]['Holiday'])

    # number formatting
    ws.cell(row=week_detail_start + num, column=2).number_format = "YYYY-MM-DD"
    ws.cell(row=week_detail_start + num, column=4).number_format = "#,##0"
    ws.cell(row=week_detail_start + num, column=5).number_format = "#,##0"

    for cell in ws[week_detail_start + num]:
        cell.alignment = Alignment(horizontal="center")

    # border formatting
    ws.cell(row=week_detail_start + num, column=1).border = extremes_border
    ws.cell(row=week_detail_start + num, column=2).border = extremes_border
    ws.cell(row=week_detail_start + num, column=3).border = extremes_border
    ws.cell(row=week_detail_start + num, column=4).border = extremes_border
    ws.cell(row=week_detail_start + num, column=5).border = extremes_border
    ws.cell(row=week_detail_start + num, column=6).border = extremes_border


week_detail_totals_row = 7 + len(week_detail) + 2

# set totals border
# totals_border = Border(
#     top=Side(border_style="thin"),
#     bottom=Side(border_style="double")
# )

ws.cell(row=week_detail_totals_row, column=3, value='Average Total')
ws.cell(row=week_detail_totals_row, column=3).alignment = Alignment(horizontal="center")
ws.cell(row=week_detail_totals_row, column=3).border = totals_border
ws.cell(row=week_detail_totals_row, column=4, value=highest_10_sum)
ws.cell(row=week_detail_totals_row, column=4).alignment = Alignment(horizontal='center')
ws.cell(row=week_detail_totals_row, column=4).number_format = "#,##0"
ws.cell(row=week_detail_totals_row, column=4).border= totals_border
ws.cell(row=week_detail_totals_row, column=5, value=highest_10_avg)
ws.cell(row=week_detail_totals_row, column=5).alignment = Alignment(horizontal='center')
ws.cell(row=week_detail_totals_row, column=5).number_format = "#,##0"
ws.cell(row=week_detail_totals_row, column=5).border= totals_border


# set column widths
ws.column_dimensions['A'].width = 10

for col in ws.columns:
    max_length = max(len(str(cell.value)) for cell in col if cell.value)
    col_letter = get_column_letter(col[0].column)
    
    if get_column_letter(col[0].column) != 'A':
        ws.column_dimensions[col_letter].width = max_length + 2


wb.save('Seoul Test.xlsx')